In [18]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. Importation des librairies nécessaires

In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


téléhargement de dataset et suppression des colonnes inutiles

In [20]:
data = pd.read_csv('https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Deep+Learning/project/spam.csv', error_bad_lines=False, encoding= 'ISO-8859-1')
data.head()

<ipython-input-20-b120372896a4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Deep+Learning/project/spam.csv', error_bad_lines=False, encoding= 'ISO-8859-1')


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [21]:
col_names = {
    'v1': "label",
    'v2': "messages"
}

data = data.rename(columns= col_names)

data.drop(["Unnamed: 2","Unnamed: 3", "Unnamed: 4"], axis = 1, inplace = True)

data.head()

,label,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [22]:
data["label"].replace({'ham':0,'spam':1}, inplace=True)
data.head()

,label,messages
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


3. Prétraitement

In [23]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['messages'])
sequences = tokenizer.texts_to_sequences(data['messages'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')


3. Séparation du dataset en ensemble d'entraînement et ensemble de test

In [24]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['label'], test_size=0.2, random_state=42)

4. Construction et entraînement d'un modèle de base de réseau de neurone dense

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=100),
    tf.keras.layers.Flatten(),                          
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           160000    
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 24)                38424     
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 198,449
Trainable params: 198,449
Non-trainable params: 0
_________________________________________________________________


In [27]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs2")

In [28]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [47]:
# Entrainement du modèle 
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, 
                    batch_size=32)


Epoch 1/10
140/140 [==============================] - 1s 10ms/step - loss: 3.3188e-04 - accuracy: 1.0000 - val_loss: 0.0939 - val_accuracy: 0.9821
Epoch 2/10
140/140 [==============================] - 1s 10ms/step - loss: 2.6266e-04 - accuracy: 1.0000 - val_loss: 0.0952 - val_accuracy: 0.9821
Epoch 3/10
140/140 [==============================] - 1s 10ms/step - loss: 2.1144e-04 - accuracy: 1.0000 - val_loss: 0.0950 - val_accuracy: 0.9821
Epoch 4/10
140/140 [==============================] - 1s 8ms/step - loss: 1.7633e-04 - accuracy: 1.0000 - val_loss: 0.0997 - val_accuracy: 0.9821
Epoch 5/10
140/140 [==============================] - 1s 5ms/step - loss: 1.4897e-04 - accuracy: 1.0000 - val_loss: 0.1018 - val_accuracy: 0.9821
Epoch 6/10
140/140 [==============================] - 1s 5ms/step - loss: 1.2365e-04 - accuracy: 1.0000 - val_loss: 0.1022 - val_accuracy: 0.9821
Epoch 7/10
140/140 [==============================] - 1s 5ms/step - loss: 1.0505e-04 - accuracy: 1.0000 - val_loss: 0.104

In [49]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

35/35 [==============================] - 0s 2ms/step - loss: 0.1076 - accuracy: 0.9821
Loss:  0.10759018361568451
Accuracy:  0.9820627570152283


5. Construction et entraînement d'un modèle de réseau neuronal récurrent (RNN)


In [39]:
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential

embedding_dim = 16
model2 = Sequential([
    Embedding(10000, embedding_dim, input_length=100),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

In [40]:
model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 16)           160000    
                                                                 
 lstm_3 (LSTM)               (None, 64)                20736     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 180,801
Trainable params: 180,801
Non-trainable params: 0
_________________________________________________________________


In [41]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
# Entrainement du modèle 
history2 = model2.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, 
                    batch_size=32)

Epoch 1/10
140/140 [==============================] - 12s 71ms/step - loss: 0.4156 - accuracy: 0.8661 - val_loss: 0.4117 - val_accuracy: 0.8655
Epoch 2/10
140/140 [==============================] - 9s 67ms/step - loss: 0.3965 - accuracy: 0.8661 - val_loss: 0.3953 - val_accuracy: 0.8655
Epoch 3/10
140/140 [==============================] - 8s 60ms/step - loss: 0.3951 - accuracy: 0.8661 - val_loss: 0.3955 - val_accuracy: 0.8655
Epoch 4/10
140/140 [==============================] - 9s 68ms/step - loss: 0.3955 - accuracy: 0.8661 - val_loss: 0.3959 - val_accuracy: 0.8655
Epoch 5/10
140/140 [==============================] - 9s 68ms/step - loss: 0.3950 - accuracy: 0.8661 - val_loss: 0.3966 - val_accuracy: 0.8655
Epoch 6/10
140/140 [==============================] - 9s 63ms/step - loss: 0.3953 - accuracy: 0.8661 - val_loss: 0.3960 - val_accuracy: 0.8655
Epoch 7/10
140/140 [==============================] - 10s 68ms/step - loss: 0.3944 - accuracy: 0.8661 - val_loss: 0.3949 - val_accuracy: 0.86

In [45]:
Résultat2 = model2.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

35/35 [==============================] - 1s 20ms/step - loss: 0.3950 - accuracy: 0.8655
Loss:  0.39495235681533813
Accuracy:  0.865470826625824


## Conclusion
Nous concluons ici que la couche Dense semble bien meilleure pour prédire la nature des messages SMS que le simple RNN. 